<a href="https://colab.research.google.com/github/yshahanov/Basic-Data-Visualization-prediction/blob/main/DCGAN_(just_for_exam_praparation).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:


import torch
import torch.nn as nn
from torchsummary import summary


class Discriminator(nn.Module):
    def __init__(self, channels_img, features_d):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            # input: N x channels_img x 64 x 64
            nn.Conv2d(channels_img, features_d, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            # _block(in_channels, out_channels, kernel_size, stride, padding)
            self._block(features_d, features_d * 2, 4, 2, 1),
            self._block(features_d * 2, features_d * 4, 4, 2, 1),
            self._block(features_d * 4, features_d * 8, 4, 2, 1),
            # After all _block img output is 4x4 (Conv2d below makes into 1x1)
            nn.Conv2d(features_d * 8, 1, kernel_size=4, stride=2, padding=0),
            nn.Sigmoid(),
        )

    def _block(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
            nn.Conv2d(
                in_channels,
                out_channels,
                kernel_size,
                stride,
                padding,
                bias=False,
            ),
            # nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2),
        )

    def forward(self, x):
        return self.disc(x)


class Generator(nn.Module):
    def __init__(self, channels_noise, channels_img, features_g):
        super(Generator, self).__init__()
        self.net = nn.Sequential(
            # Input: N x channels_noise x 1 x 1
            self._block(channels_noise, features_g * 16, 4, 1, 0),  # img: 4x4
            self._block(features_g * 16, features_g * 8, 4, 2, 1),  # img: 8x8
            self._block(features_g * 8, features_g * 4, 4, 2, 1),  # img: 16x16
            self._block(features_g * 4, features_g * 2, 4, 2, 1),  # img: 32x32
            nn.ConvTranspose2d(
                features_g * 2, channels_img, kernel_size=4, stride=2, padding=1
            ),
            # Output: N x channels_img x 64 x 64
            nn.Tanh(),
        )

    def _block(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
            nn.ConvTranspose2d(
                in_channels,
                out_channels,
                kernel_size,
                stride,
                padding,
                bias=False,
            ),
            # nn.BatchNorm2d(out_channels),
            nn.ReLU(),
        )

    def forward(self, x):
        return self.net(x)






In [ ]:
disc = Discriminator(3, 24)

print(summary(disc, (3, 256, 256)))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 24, 128, 128]           1,176
         LeakyReLU-2         [-1, 24, 128, 128]               0
            Conv2d-3           [-1, 48, 64, 64]          18,432
         LeakyReLU-4           [-1, 48, 64, 64]               0
            Conv2d-5           [-1, 96, 32, 32]          73,728
         LeakyReLU-6           [-1, 96, 32, 32]               0
            Conv2d-7          [-1, 192, 16, 16]         294,912
         LeakyReLU-8          [-1, 192, 16, 16]               0
            Conv2d-9              [-1, 1, 7, 7]           3,073
          Sigmoid-10              [-1, 1, 7, 7]               0
Total params: 391,321
Trainable params: 391,321
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.75
Forward/backward pass size (MB): 11.25
Params size (MB): 1.49
Estimated 

In [ ]:
gen = Generator(1,3,24)

print(summary(gen, (1, 32, 32)))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
   ConvTranspose2d-1          [-1, 384, 35, 35]           6,144
              ReLU-2          [-1, 384, 35, 35]               0
   ConvTranspose2d-3          [-1, 192, 70, 70]       1,179,648
              ReLU-4          [-1, 192, 70, 70]               0
   ConvTranspose2d-5         [-1, 96, 140, 140]         294,912
              ReLU-6         [-1, 96, 140, 140]               0
   ConvTranspose2d-7         [-1, 48, 280, 280]          73,728
              ReLU-8         [-1, 48, 280, 280]               0
   ConvTranspose2d-9          [-1, 3, 560, 560]           2,307
             Tanh-10          [-1, 3, 560, 560]               0
Total params: 1,556,739
Trainable params: 1,556,739
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 122.02
Params size (MB): 5.94
Estim

In [ ]:
def initialize_weights(model):
    # Initializes weights according to the DCGAN paper
    for m in model.modules():
        if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d, nn.BatchNorm2d)):
            nn.init.normal_(m.weight.data, 0.0, 0.02)


def test():
    N, in_channels, H, W = 8, 3, 64, 64
    noise_dim = 100
    x = torch.randn((N, in_channels, H, W))
    disc = Discriminator(in_channels, 8)
    assert disc(x).shape == (N, 1, 1, 1), "Discriminator test failed"
    gen = Generator(noise_dim, in_channels, 8)
    z = torch.randn((N, noise_dim, 1, 1))
    assert gen(z).shape == (N, in_channels, H, W), "Generator test failed"
    print("Success, tests passed!")

initialize_weights(gen)
initialize_weights(disc)
test()

Success, tests passed!
